In [1]:
%matplotlib inline

import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/train.csv")

In [3]:
df.drop(["ID", "lead_time", "arrival_date_week_number", "country", "market_segment", "distribution_channel",
              "is_repeated_guest", "previous_cancellations", "previous_bookings_not_canceled", "reserved_room_type",
             "assigned_room_type", "booking_changes", "deposit_type", "agent", "company", "days_in_waiting_list",
             "customer_type", "required_car_parking_spaces", "total_of_special_requests"], axis=1, inplace = True)

In [4]:
def create_datetime(year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
    return pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")

In [5]:
df["date"] = create_datetime(df["arrival_date_year"], df["arrival_date_month"], df["arrival_date_day_of_month"])
# df.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "reservation_status_date"], 
#         axis=1, inplace=True)
df.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], 
        axis=1, inplace=True)

In [7]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], format="%Y-%m-%d") - timedelta(days=1)

In [8]:
%%time

df['date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(df['date']), pd.to_datetime(df['reservation_status_date']))]
# df = df.explode('date').drop(['reservation_status_date', 'date'], axis=1)

CPU times: user 11.6 s, sys: 98.3 ms, total: 11.7 s
Wall time: 11.7 s


In [9]:
df = df.explode("date").drop(["reservation_status_date"], axis=1).reset_index(drop=True)

In [10]:
df["stay_nights"] = df["stays_in_weekend_nights"] + df["stays_in_week_nights"] 

In [35]:
traindf = df[df["stay_nights"] != 0]
traindf = traindf.drop(["stays_in_week_nights", "stays_in_weekend_nights", "is_canceled"], axis=1)
traindf = traindf.dropna().sort_values(by=['date'])

In [17]:
targetdf = pd.read_csv("../data/train_label.csv")

In [19]:
pd.merge(traindf, targetdf, how="inner", left_on="", right_on="arrival_date")

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [41]:
traindf.groupby("date").apply(lambda x: sum(x["adults"] + x["children"] + x["babies"]))

date
2015-07-01    182.0
2015-07-02    246.0
2015-07-03    190.0
2015-07-04    268.0
2015-07-05    277.0
              ...  
2017-04-09     13.0
2017-04-10      6.0
2017-04-11      4.0
2017-04-12      1.0
2017-04-13      1.0
Length: 653, dtype: float64